In [1]:
import os

In [2]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    update_base_model_path: Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_dir: Path

In [6]:
from PizzaSteakClassifier.constants import *
from PizzaSteakClassifier.utils.common import read_yaml_file, create_directories
import tensorflow as tf

[2024-01-05 22:33:52,169 - WARNING - module_wrapper - From h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config

        create_directories([
            Path(config.prepare_callbacks.tensorboard_log_dir),
            Path(config.prepare_callbacks.checkpoint_dir)
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.artifacts_root),
            tensorboard_log_dir = Path(config.prepare_callbacks.tensorboard_log_dir),
            checkpoint_dir = Path(config.prepare_callbacks.checkpoint_dir)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.model_training
        params = self.params
        base_model_preparation = self.config.base_model_preparation
        training_dir = os.path.join(self.config.data_ingestion.unzip_dir)

        create_directories([training.root_dir])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            update_base_model_path=Path(base_model_preparation.updated_base_model_path),
            training_data=Path(training_dir),
            params_epoch=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time


In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config
    
    @property
    def create_tb_callback(self):
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_log_dir,
            f"tb_logs_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_dir), #str is required for ModelCheckpoint
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
                self.create_tb_callback,
                self.create_ckpt_callback
        ]

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.update_base_model_path
        )
    
    def train_valid_generator(self):
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
        data_flow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )
        self.valid_data_generator = valid_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **data_flow_kwargs
        )
        if self.config.params_is_augmentation:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **data_generator_kwargs
            )
        else:
            train_data_generator = valid_data_generator

        self.train_data_generator = train_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **data_flow_kwargs
        )
    
    @staticmethod
    def save_model(model: tf.keras.Model, model_path: Path):
        model.save(model_path)
    
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_data_generator.samples // self.train_data_generator.batch_size
        self.valid_steps = self.valid_data_generator.samples // self.valid_data_generator.batch_size

        self.model.fit(
            self.train_data_generator,
            steps_per_epoch = self.steps_per_epoch,
            epochs = self.config.params_epoch,
            validation_data = self.valid_data_generator,
            validation_steps = self.valid_steps,
            callbacks = callback_list
        )

        self.save_model(
            model = self.model,
            model_path = self.config.trained_model_path
        )

In [11]:
print(tf.__version__)

2.15.0


In [16]:
try:
    config=ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callback_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)
    
except Exception as e:
    raise e


[2024-01-05 22:41:44,612 - INFO - common - YAML file loaded successfully: config\config.yaml]
[2024-01-05 22:41:44,614 - INFO - common - YAML file loaded successfully: params.yaml]
[2024-01-05 22:41:44,615 - INFO - common - Directory created: artifacts]
[2024-01-05 22:41:44,617 - INFO - common - Directory created: artifacts\prepare_callbacks\tensorboard_logs]
[2024-01-05 22:41:44,619 - INFO - common - Directory created: artifacts\prepare_callbacks\checkpoint_dir]
[2024-01-05 22:41:44,621 - INFO - common - Directory created: artifacts/model_training]
Found 400 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.
Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\traitlets\config\application.py", line 1077, in launch_instance

  File "h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\heispv\AppData\Local\Programs\Python\Python311\Lib\asyncio\windows_events.py", line 321, in run_forever

  File "C:\Users\heispv\AppData\Local\Programs